# Install requerements

In [1]:
import requests
from tabula import read_pdf
import pandas as pd
from pathlib import Path

# Download the PDF

In [132]:
pdf_path = Path("/Users/marco/Library/CloudStorage/OneDrive-UniversitàdegliStudidiMilano/PersDocs/Banking-Financial/RendicontazioneAnnuale/2024/TradeRepublic/TR-Spet23-Lug24.pdf")
pdf_folder = pdf_path.parent

view images

In [133]:
from pdf2image import convert_from_path
import numpy as np
from matplotlib import pyplot as plt

images_of_pdf = convert_from_path(pdf_path, dpi=200)  # Convert PDF to List of PIL Images
imgs_path = []

for i,page in enumerate(images_of_pdf):
    #page.save(str(pdf_folder) + "/page_image_"+str(i)+".jpg")
    imgs_path.append(str(pdf_folder) + "/page_image_"+str(i)+".jpg")

readable_images_of_pdf = []  # Create a list for thr for loop to put the images into
for PIL_Image in images_of_pdf:
    readable_images_of_pdf.append(np.array(PIL_Image))  # Add items to list

n_pages = len(images_of_pdf)
print("n pages exported: ", n_pages)
type(readable_images_of_pdf)

n pages exported:  6


list

let's load the images to have a preview of the modifications we will do for tabula

In [134]:
#from google.colab.patches import cv2_imshow
import cv2

#reload the img in cv2
imgs= []
for i,page in enumerate(imgs_path):
    imgs.append(cv2.imread(page, cv2.IMREAD_UNCHANGED))


In [136]:
# Display the image
cv2.imshow('Image', imgs[0])
# Wait for 1 millisecond
cv2.waitKey(1)

-1

In [137]:
# Close the window
cv2.destroyWindow('Image')
cv2.waitKey(1) #this will ensure the closure of the event loop and prevent the window to reopen

-1

In [139]:
#resize the img to the original values of the pdf. To get them open it with any pdf reader and get those info or look in the file properties. Values are in pts.
for i, img in enumerate(imgs):
    imgs[i] = cv2.resize(img, (int(595.28), int(841.88)), interpolation=cv2.INTER_LINEAR)

# Get the dimensions of the image
height, width, channels = img.shape

print(f"Width: {width}, Height: {height}, Channels: {channels}")

Width: 595, Height: 841, Channels: 3


In [174]:
# Define the x points for vertical lines
x_points = [80, 148, 435, 475, 507, 555]  # Example x points

# Define the area to color (top, left, bottom, right)
area = [82,42.042,790.606,556.959]
area_p1 = [280.295,42.042,789.118,552.494]  #the area for the first page
area_last = [82,42.042,790.606,556.959] #the area for the last page where u have a final piece of text

In [179]:
#MODIFY THIS
page_number = 3

#define the right area
if page_number == 0:
    area_preview = area_p1
elif page_number == n_pages-1:
    area_preview = area_last
else:
    area_preview = area

#reset the image every time
# img = cv2.imread("/content/page_image_0.jpg", cv2.IMREAD_UNCHANGED)
# img = cv2.resize(img, (int(595.28), int(841.88)), interpolation=cv2.INTER_LINEAR)
img = imgs[page_number].copy()

overlay = np.ones((img.shape[0], img.shape[1], 4), dtype='uint8')

# Convert the original image to BGRA (with alpha channel)
img_bgra = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)

# Color the defined area
cv2.rectangle(overlay, (int(area_preview[1]), int(area_preview[0])), (int(area_preview[3]), int(area_preview[2])), (255, 0, 0, 125), -1)  # Blue filled rectangle

# Draw vertical lines
for x in x_points:
    cv2.line(overlay, (x, 0), (x, img.shape[0]), (0, 255, 0,255), 2)  # Green vertical lines

# Blend the overlay with the original image
combined = cv2.addWeighted(img_bgra, 0.9, overlay, 0.2, 0)

# Display the image
cv2.imshow('overlay', combined)
cv2.waitKey(1)

-1

In [180]:
# Close the window
cv2.destroyWindow('overlay')
cv2.waitKey(1) #this will ensure the closure of the event loop and prevent the window to reopen

-1

# Extract tables

In [181]:
pandas_options = {
    'header': None,
    'columns' : ["DATA", "TIPO", "DESCRIZIONE", "IN ENTRATA","IN USCITA", "SALDO"]
}

Inspect the pdf, if in the last page u don't have any data, execute following cell and change the value of **extract_last_page**

In [182]:
extract_last_page = False
if not extract_last_page:
    n_pages = n_pages-1

In [183]:
with open(pdf_path, 'rb') as file:
        # extract table from pages
        dfs = read_pdf(
            file,
            pages="2-" + str(n_pages), # pages to extract table from
            area=area,
            columns=x_points,
            #lattice=True,
            pandas_options=pandas_options,
            stream=True,
        )

        # extract table in page 1
        dfs_1 = read_pdf(
            file,
            pages="1",
            area=area_p1,
            columns=x_points,
            pandas_options=pandas_options,
            stream = True,
        )

        if extract_last_page:
            #extract table in last page
            dfs_2 = read_pdf(
                file,
                pages=str(n_pages),
                area=area_last,
                columns=x_points,
                pandas_options=pandas_options,
                stream = True,
            )
print(len(dfs))
dfs[0]
#dfs_1[0]
#dfs_2[0]

/Users/marco/miniconda3/envs/docpdf_env/lib/python3.12/site-packages/tabula/io.py:1054: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


3


/Users/marco/miniconda3/envs/docpdf_env/lib/python3.12/site-packages/tabula/io.py:1054: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


,DATA,TIPO,DESCRIZIONE,IN ENTRATA,IN USCITA,SALDO
0,25 mag,Transazione con,NaN,NaN,NaN,NaN
1,NaN,NaN,TooGoodT cw9t43xst9130,NaN,"3,99 €","4.321,02 €"
2,2024,carta,NaN,NaN,NaN,NaN
3,26 mag,Transazione con,NaN,NaN,NaN,NaN
4,NaN,NaN,WWW.SOC.CHIM.IT VBV,NaN,"60,00 €","4.261,02 €"
...,...,...,...,...,...,...
61,NaN,NaN,SUSHI HIRO SAS DI XIANG,NaN,"63,80 €","3.170,94 €"
62,2024,carta,NaN,NaN,NaN,NaN
63,10 giu,Transazione con,NaN,NaN,NaN,NaN
64,NaN,NaN,TIGROS 2,NaN,"86,56 €","3.084,38 €"


let's make a single df object with all the exported tables

In [203]:
single_df = pd.concat([df.iloc[:] for df in dfs[:]], axis=0, ignore_index=True)
single_df = pd.concat([single_df, dfs_1[0].iloc[:]], axis=0, ignore_index=True)
if extract_last_page:
    single_df = pd.concat([single_df, dfs_2[0].iloc[:]], axis=0, ignore_index=True)
single_df

,DATA,TIPO,DESCRIZIONE,IN ENTRATA,IN USCITA,SALDO
0,25 mag,Transazione con,NaN,NaN,NaN,NaN
1,NaN,NaN,TooGoodT cw9t43xst9130,NaN,"3,99 €","4.321,02 €"
2,2024,carta,NaN,NaN,NaN,NaN
3,26 mag,Transazione con,NaN,NaN,NaN,NaN
4,NaN,NaN,WWW.SOC.CHIM.IT VBV,NaN,"60,00 €","4.261,02 €"
...,...,...,...,...,...,...
241,NaN,NaN,TIGROS 17,NaN,"5,31 €","4.810,01 €"
242,2024,carta,NaN,NaN,NaN,NaN
243,25 mag,Transazione con,NaN,NaN,NaN,NaN
244,NaN,NaN,KEYNOTE CONFERENCE SER,NaN,"485,00 €","4.325,01 €"


If column labels are present in the df, run following cell

In [205]:
# Drop rows if they contain all column names
try:
    single_df = single_df.drop(single_df[single_df.apply(lambda row: all(item in row.values for item in pandas_options['columns']), axis=1)].index)
    # Reset the index
    single_df = single_df.reset_index(drop=True)
except KeyError:
    ("everything ok")
single_df

,DATA,TIPO,DESCRIZIONE,IN ENTRATA,IN USCITA,SALDO
0,25 mag,Transazione con,NaN,NaN,NaN,NaN
1,NaN,NaN,TooGoodT cw9t43xst9130,NaN,"3,99 €","4.321,02 €"
2,2024,carta,NaN,NaN,NaN,NaN
3,26 mag,Transazione con,NaN,NaN,NaN,NaN
4,NaN,NaN,WWW.SOC.CHIM.IT VBV,NaN,"60,00 €","4.261,02 €"
...,...,...,...,...,...,...
241,NaN,NaN,TIGROS 17,NaN,"5,31 €","4.810,01 €"
242,2024,carta,NaN,NaN,NaN,NaN
243,25 mag,Transazione con,NaN,NaN,NaN,NaN
244,NaN,NaN,KEYNOTE CONFERENCE SER,NaN,"485,00 €","4.325,01 €"


At this point u should a have a df in which each row is actually divided over 3 consequtive rows. If not, the following cell will return a message and u should inspect your pdf/df with the help of the group subdivision printed inside the df object.

In [212]:
if single_df.index.stop % 3 == 0:
    print("All good")
else:
    print("Error, u should inspect ur pdf/df object")
    single_df['group'] = single_df.index // 3

All good


In [213]:
# Group the DataFrame by every 3 rows as tabula didn't recognize multiline text in the cells. U should inspect this by urself
grouped = single_df.groupby(single_df.index // 3)

# Aggregate the text within each group
result = grouped.agg(lambda x: ' '.join(x.fillna(' ')))

# Reset the index
result = result.reset_index(drop=True)

result

,DATA,TIPO,DESCRIZIONE,IN ENTRATA,IN USCITA,SALDO
0,25 mag 2024,Transazione con carta,TooGoodT cw9t43xst9130,,"3,99 €","4.321,02 €"
1,26 mag 2024,Transazione con carta,WWW.SOC.CHIM.IT VBV,,"60,00 €","4.261,02 €"
2,26 mag 2024,Transazione con carta,TooGoodT 3gx6rfhz6xah0,,"4,99 €","4.256,03 €"
3,26 mag 2024,Transazione con carta,ASPIT DIREZ. USCITA -,,"1,60 €","4.254,43 €"
4,26 mag 2024,Transazione con carta,ASPIT DIREZ.ENTRATA -,,"1,60 €","4.252,83 €"
...,...,...,...,...,...,...
77,21 mag 2024,Transazione con carta,PAOLO & VALE S.R.L.S.,,"104,80 €","4.891,01 €"
78,21 mag 2024,Transazione con carta,AUTOGRILL 0736,,"9,40 €","4.881,61 €"
79,22 mag 2024,Transazione con carta,PV ENI - 03970,,"66,29 €","4.815,32 €"
80,24 mag 2024,Transazione con carta,TIGROS 17,,"5,31 €","4.810,01 €"


# Export data to CSV

In [214]:
result.to_csv(str(pdf_folder) + "/" + pdf_path.stem + "output.csv", index=False)